In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import img_to_array, load_img

### Load the dataset 

In [2]:
dataset_path = '../../data/preprocessed_dataset.csv'
df = pd.read_csv(dataset_path)

In [3]:
df

,Image,Label
0,..\data\Fish_Dataset\Black Sea Sprat\Black Sea...,Black Sea Sprat
1,..\data\Fish_Dataset\Black Sea Sprat\Black Sea...,Black Sea Sprat
2,..\data\Fish_Dataset\Black Sea Sprat\Black Sea...,Black Sea Sprat
3,..\data\Fish_Dataset\Black Sea Sprat\Black Sea...,Black Sea Sprat
4,..\data\Fish_Dataset\Black Sea Sprat\Black Sea...,Black Sea Sprat
...,...,...
8995,..\data\Fish_Dataset\Trout\Trout\00996.png,Trout
8996,..\data\Fish_Dataset\Trout\Trout\00997.png,Trout
8997,..\data\Fish_Dataset\Trout\Trout\00998.png,Trout
8998,..\data\Fish_Dataset\Trout\Trout\00999.png,Trout


### Split the data into features (X) and labels (y)

In [4]:
X = df['Image']
y = df['Label']

In [5]:
X[0], y[0]

('..\\data\\Fish_Dataset\\Black Sea Sprat\\Black Sea Sprat\\00001.png',
 'Black Sea Sprat')

### Train and Test data

In [6]:
# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Image Preprocessing

In [7]:
# Preprocess train and test images
def preprocess_images(image_paths, target_size=(224, 224)):
    images = []
    for path in image_paths:
        img = load_img(path, target_size=target_size)
        img_array = img_to_array(img)
        images.append(img_array)
    return np.array(images)

In [ ]:
# Preprocess train and test images
X_train = preprocess_images(X_train.values)
X_test = preprocess_images(X_test.values)

In [ ]:
# Normalize pixel values to be between 0 and 1
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
# Convert labels to categorical one-hot encoding
y_train = to_categorical(y_train, num_classes=8)
y_test = to_categorical(y_test, num_classes=8)

### CNN Model

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(8, activation='softmax'))

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Train Model

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

### Evaluate the model on the test set

In [ ]:
# Evaluate the model on the test set
predictions = model.predict(X_test)
y_pred = np.argmax(predictions, axis=1)
y_true = np.argmax(y_test, axis=1)

### Performance

In [ ]:
# Calculate accuracy, confusion matrix, and classification report
accuracy = np.mean(y_pred == y_true)
conf_matrix = confusion_matrix(y_true, y_pred)
class_report = classification_report(y_true, y_pred)

In [ ]:
print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Classification Report:\n{class_report}")

### Graphs

In [ ]:
# Plot confusion matrix
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=[f'Class {i}' for i in range(8)],
            yticklabels=[f'Class {i}' for i in range(8)])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

### Save Model

In [ ]:
model.save('cnn_model.h5')